## Import Libraries

In [33]:
from collections import defaultdict
import copy
import random
import os
import sys
import shutil
from urllib.request import urlretrieve
#from imutils import paths #Python library that provides a collection of convenience functions for common tasks in computer vision and image processing
#from sklearn.model_selection import train_test_split  #An open-source Python library that provides a wide range of tools and algorithms for machine learning, data mining, and data analysis.
import json
import importlib
import logging
import gc

from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np


import albumentations as A
from albumentations.pytorch import ToTensorV2


import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn #All neural network modules, nn.linear, nn.conv2d, BatchNorm, Loss functions
import torch.optim as optim #For all optimization algorithms SGD, Adam
import torch.nn.functional as F #All functions that dont have any parameter
from torch.utils.data import Dataset, DataLoader #Pytorch standard dataset and its management
from torchsummary import summary

from tqdm import tqdm

cudnn.benchmark = True

## Check Pytorych Installation

In [34]:
#Check Pytorch Installation
print(torch.version.__version__, torch.cuda.is_available())

2.1.2+cu118 True


## Load the test dataset

In [35]:
# Loading the images and masks test dataset
test_dataset_root_directory = os.path.join(os.environ["HOME"], "Segmentation_Project/Crack_Datasets/Crack500_cropped/Test") 

test_images_dataset_path = os.path.join(test_dataset_root_directory, "Images") # provide the test images path
test_masks_dataset_path = os.path.join(test_dataset_root_directory, "Masks")    # provide the test masks path

test_images_filenames = list(sorted(os.listdir(test_images_dataset_path)))
print("Testing dataset size:", len(test_images_filenames))

Testing dataset size: 2000


In [36]:
#Check the pixel values of the mask image:
for i, image_filename in enumerate(test_images_filenames[:5]):
        
        test_mask_path = os.path.join(test_masks_dataset_path, image_filename.replace(".jpg", ".png")) #mask path
        test_mask = np.array(Image.open(test_mask_path).convert("L"), dtype=np.float32) #open masks from the mask path specified
        print("Unique pixel values in train mask:", np.unique(test_mask))

Unique pixel values in train mask: [  0. 255.]
Unique pixel values in train mask: [  0. 255.]
Unique pixel values in train mask: [  0. 255.]
Unique pixel values in train mask: [  0. 255.]
Unique pixel values in train mask: [  0. 255.]


## Define the Pytorch Test Dataset

In [37]:
class SegmentationInferenceDataset(Dataset):
    def __init__(self, images_filename, images_directory, masks_directory, transform=None):
        # store the images filenames, images path, masks path, augmentations transform
        self.images_filenames = images_filename
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform

    def __len__(self):
        # return the number of total samples contained in the dataset
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image_path = os.path.join(self.images_directory, image_filename)  # provide the image path
        mask_path = os.path.join(self.masks_directory, image_filename.replace(".jpg", ".png")) # provide the mask  path

        image = np.array(Image.open(image_path).convert("RGB")) #open images from image path provided (PIL format required by Pytorch)
        #original_size = tuple(image.shape[:2]) #we need original sizes if we resize the input image

        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32) #open masks from the mask path provided (PIL format required by Pytorch)
        #Preprocess mask
        mask[mask == 255.0] = 1.0 # for binary segmentation convert the 255 pixel values to 1, sigmoid[0-1] as last activation layer

        # check to see if we are applying any transformations
        if self.transform is not None:
           augmentations = self.transform(image=image, mask=mask)
           image = augmentations["image"] # apply the transformations to image
           mask = augmentations["mask"] #apply the transformation to mask

        # return a tuple of images (with original size if resize) and masks
        return image, mask

In [38]:
test_transform = A.Compose(
    [
        #A.Resize(height = 288, width = 512, interpolation=cv2.INTER_NEAREST), 
        A.Normalize(
            mean=[0.0, 0.0, 0.0], 
            std = [1.0,1.0,1.0],
            max_pixelvalues=255.0
        ),
        ToTensorV2()
    ]
)
test_dataset = SegmentationInferenceDataset(test_images_filenames, test_images_dataset_path, test_masks_dataset_path, transform=test_transform,)
print(len(test_dataset))

2000


## Define the evaluation metrics

In [39]:
# Evaluation Metrics
def dice_score(pred, target, threshold, eps=1e-15): # --> main metric to monitor for validation (it corresponds to F-1 score)
    pred = (pred > threshold).float() #binarize predictions based on the given threshold
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    dice_score = (2. * intersection + eps) / (union + eps)
    return dice_score

def Jaccard_score(pred, target, threshold, eps=1e-15):
    pred = (pred > threshold).float() #binarize predictions based on the given threshold
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    Jaccard_score = (intersection + eps) / (union + eps)
    return Jaccard_score

def IOU_score(pred, target, threshold):
    pred = (pred > threshold).float() #binarize predictions based on the given threshold
    tp = ((pred == 1) & (target == 1)).sum() # True Positives (TP): Both pred and target are 1
    fp = ((pred == 1) & (target == 0)).sum() # False Positives (FP): pred is 1, target is 0
    fn = ((pred == 0) & (target == 1)).sum()  # False Negatives (FN): pred is 0, target is 1
    IOU_score = tp / (tp + fp + fn)
    return IOU_score

def accuracy(pred, target, threshold):
    pred = (pred > threshold).float() #binarize predictions based on the given threshold
    num_correct = (pred == target).sum()
    num_pixels = torch.numel(target)
    accuracy = num_correct/num_pixels
    return accuracy*100

def precision(pred, target, threshold, eps=1e-15):
    pred = (pred > threshold).float() #binarize predictions based on the given threshold
    tp = ((pred == 1) & (target == 1)).sum() # True Positives (TP): Both pred and target are 1
    fp = ((pred == 1) & (target == 0)).sum() # False Positives (FP): pred is 1, target is 0
    precision = (tp + eps) / (tp + fp + eps) # eps (very small) to avoid nan (division by zero as observed for some models)
    return precision

## Define the inference function

In [40]:
def inference(model, test_dataset, params):
    
    test_data_loader = DataLoader(
        test_dataset,
        batch_size=params["batch_size"],
        num_workers=params["num_workers"],
        shuffle=False,
        pin_memory=True,
    )

    test_running_accuracy = 0
    test_running_precision = 0
    test_running_dice_score = 0
    test_running_IOU_score = 0
    test_dataset_steps = len(test_dataset) // params["batch_size"]

    inference_output = []

    model.eval () 
    #with torch.no_grad():

    for batch_idx, (images, targets) in enumerate(test_data_loader, start=1):

        #print(f"Processing batch: {batch_idx}") #print the processing batch to keep track
            

        images = images.to(params["device"], non_blocking =True) #load the images in a batch and move to gpu for computation
        targets = targets.float().unsqueeze(1).to(params["device"], non_blocking=True) #load the targets mask, add the batch dimension, convert to float point and move it to GPU for computation

        # Extract original dimensions from `original_sizes` if resize the input image and mask
        #original_heights, original_widths = original_sizes
        #print(f"Original_heights: {original_heights}") #should be a tensor of batch size
        #print(f"Original_widths: {original_widths}") #should be a tensor of batch size

        #forward propagation
        with torch.no_grad():

            if params["model"] == "DcsNet_deep":
                predictions, deep_sup_1, deep_sup_2 = model(images)
            else:
                predictions = model(images) #make predictions on the batch of test images
        
            prediction_probabilities = torch.sigmoid(predictions.squeeze(1)) #converting the predictions to probability and removing the channel dimension as gray scale

            #After making the predictions calculate the accuracy, precision, dice_score and IOU_score as evaluation metrics for the current batch of the test images
            accuracy_value = accuracy(torch.sigmoid(predictions), targets, params["threshold"])
            precision_value = precision(torch.sigmoid(predictions), targets, params["threshold"])
            dice_score_value = dice_score(torch.sigmoid(predictions), targets, params["threshold"])
            IOU_score_value = IOU_score(torch.sigmoid(predictions), targets, params["threshold"])

            #update the evaluation metrics for each batch
            test_running_accuracy += accuracy_value.item()
            test_running_precision += precision_value.item()
            test_running_dice_score += dice_score_value.item()
            test_running_IOU_score += IOU_score_value.item()

            #store the predictions with original_height and original width (if resize the image and mask) for visualization
            predicted_masks = (prediction_probabilities > params["threshold"]).float() * 1.0 # Binary thresholding
            predicted_masks = predicted_masks.cpu().numpy()  # Convert to NumPy array and shift to cpu memory
            
            # Append predictions with correct handling
            for predicted_mask in predicted_masks:
                
                #print(f"Predicted_mask_size: {predicted_mask.shape}")
                
                inference_output.append((predicted_mask))  # Correctly append to list
                

    #calculate the average metrics           
    test_accuracy = test_running_accuracy / test_dataset_steps #average test accuracy for the entire test dataset
    test_precision = test_running_precision / test_dataset_steps #average test precision for the entire test dataset
    test_dice_score = test_running_dice_score / test_dataset_steps #average test dice score for the entire test dataset
    test_IOU_score = test_running_IOU_score / test_dataset_steps #average test jaccard score for the entire test dataset
    
    #Create a dictionary for evaluation metrics
    eval_metrics = {
        "accuracy": test_accuracy,
        "precision": test_precision,
        "dice_score": test_dice_score,
        "IOU_score": test_IOU_score
    }

    print("Test Accuracy: {:.4f}, Test Precision: {:.4f}, Test Dice Score: {:.4f}, Test IOU Score: {:.4f}".format(test_accuracy, test_precision, test_dice_score, test_IOU_score))

    return eval_metrics, inference_output

## Function to load the trained models and do the inference

In [41]:
# Define the root directory to save trained models
model_save_root_directory = "/home/ali/Segmentation_Project/Outputs/Trained_Models"

def load_model_and_inference(model_name, test_dataset, test_params):

    #load the trained model
    trained_model = torch.load(os.path.join(model_save_root_directory, f"{model_name}")).to(test_params["device"]) #load the model and transfer it to gpu
    print(f"Trained {model_name} model is loaded successfully...")

    #make inference using the trained model
    eval_metrics, inference_output = inference(trained_model, test_dataset, test_params)

    return eval_metrics, inference_output

## Perform Inference ResNet-UNet models

In [42]:
#Define test parameters
params = {
    "model": "ResNetUNet",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 16, #same as used for training and validation
    "num_workers": 2, #same as used for training and validation
    "threshold": 0.5
}

metric_ResNetUNet18, infer_ResNetUNet18 = load_model_and_inference("ResUNet18.pth", test_dataset, params) #ResUNet18_2 model
metric_ResNetUNet18_AM, infer_ResNetUNet18_AM = load_model_and_inference("ResUNet18_AM.pth", test_dataset, params) #ResUNet18_AM model
metric_ResNetUNet34, infer_ResNetUNet34 = load_model_and_inference("ResUNet34_2.pth", test_dataset, params) #ResUNet34 model
metric_ResNetUNet34_AM, infer_ResNetUNet34_AM = load_model_and_inference("ResUNet34_AM.pth", test_dataset, params) #ResUNet34_AM model

/home/ali/miniconda3/envs/seg_prac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trained ResUNet18.pth model is loaded successfully...
Test Accuracy: 95.6144, Test Precision: 0.7690, Test Dice Score: 0.7269, Test IOU Score: 0.5805
Trained ResUNet18_AM.pth model is loaded successfully...
Test Accuracy: 95.6436, Test Precision: 0.7639, Test Dice Score: 0.7324, Test IOU Score: 0.5869
Trained ResUNet34_2.pth model is loaded successfully...
Test Accuracy: 95.6957, Test Precision: 0.7604, Test Dice Score: 0.7383, Test IOU Score: 0.5939
Trained ResUNet34_AM.pth model is loaded successfully...
Test Accuracy: 95.7251, Test Precision: 0.7619, Test Dice Score: 0.7400, Test IOU Score: 0.5960


## Perform Inference EfficientNet-UNet Models

In [43]:
#Define test parameters
params = {
    "model": "EfficientNetUNet",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 16, #same as used for training and validation
    "num_workers": 2, #same as used for training and validation
    "threshold": 0.5
}

#metric_EffNetUNetB1, infer_EffNetUNetB1 = load_model_and_inference("EffNetUNetB1_2.pth", test_dataset, params) #EffNetUNetB1 model
metric_EffNetUNetB1_AM, infer_EffNetUNetB1_AM = load_model_and_inference("EffNetUNetB1_AM_2.pth", test_dataset, params) #EffNetUNetB1_AM model
#metric_EffNetUNetB2, infer_EffNetUNetB2 = load_model_and_inference("EffNetUNetB2.pth", test_dataset, params) #EffNetUNetB2 model
metric_EffNetUNetB2_AM, infer_EffNetUNetB2_AM = load_model_and_inference("EffNetUNetB2_AM.pth", test_dataset, params) #EffNetUNetB2_AM model
#metric_EffNetUNetB3, infer_EffNetUNetB3 = load_model_and_inference("EffNetUNetB3_2.pth", test_dataset, params) #EffNetUNetB3 model
metric_EffNetUNetB3_AM, infer_EffNetUNetB3_AM = load_model_and_inference("EffNetUNetB3_AM_2.pth", test_dataset, params) #EffNetUNetB3_AM model
#metric_EffNetUNetB4, infer_EffNetUNetB4 = load_model_and_inference("EffNetUNetB4_2.pth", test_dataset, params) #EffNetUNetB3 model
metric_EffNetUNetB4_AM, infer_EffNetUNetB4_AM = load_model_and_inference("EffNetUNetB4_AM_2.pth", test_dataset, params) #EffNetUNetB3_AM model


Trained EffNetUNetB1_AM_2.pth model is loaded successfully...
Test Accuracy: 95.5134, Test Precision: 0.7617, Test Dice Score: 0.7242, Test IOU Score: 0.5766
Trained EffNetUNetB2_AM.pth model is loaded successfully...
Test Accuracy: 95.4996, Test Precision: 0.7653, Test Dice Score: 0.7222, Test IOU Score: 0.5740
Trained EffNetUNetB3_AM_2.pth model is loaded successfully...
Test Accuracy: 95.5243, Test Precision: 0.7570, Test Dice Score: 0.7278, Test IOU Score: 0.5805
Trained EffNetUNetB4_AM_2.pth model is loaded successfully...
Test Accuracy: 95.4210, Test Precision: 0.7428, Test Dice Score: 0.7262, Test IOU Score: 0.5787


## Perform Inference MANet models

In [44]:
#Define test parameters
params = {
    "model": "ResNetMANet",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 16, #same as used for training and validation
    "num_workers": 2, #same as used for training and validation
    "threshold": 0.5
}

metric_ResMANet18, infer_ResMANet18 = load_model_and_inference("ResMANet18.pth", test_dataset, params) #ResNetMANet model
metric_ResMANet34, infer_ResMANet34 = load_model_and_inference("ResMANet34.pth", test_dataset, params) #ResNetMANet model

Trained ResMANet18.pth model is loaded successfully...


Test Accuracy: 95.5892, Test Precision: 0.7644, Test Dice Score: 0.7284, Test IOU Score: 0.5814
Trained ResMANet34.pth model is loaded successfully...
Test Accuracy: 95.7076, Test Precision: 0.7638, Test Dice Score: 0.7374, Test IOU Score: 0.5927


## Perform Inference DcsNet Models

In [45]:
#Define test parameters
params = {
    "model": "DcsNet",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 16, #same as used for training and validation
    "num_workers": 2, #same as used for training and validation
    "threshold": 0.5
}

metric_DcsNet, infer_DcsNet = load_model_and_inference("DcsNet_2.pth", test_dataset, params) #DcsNet model

Trained DcsNet_2.pth model is loaded successfully...


Test Accuracy: 95.7153, Test Precision: 0.7737, Test Dice Score: 0.7337, Test IOU Score: 0.5879


## Perform Inference UTNet Models

In [14]:
#Define test parameters
params = {
    "model": "UTNet",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 16, #same as used for training and validation
    "num_workers": 2, #same as used for training and validation
    "threshold": 0.5
}

metric_UTNet, infer_UTNet = load_model_and_inference("UTNet.pth", test_dataset, params) #DcsNet model

Trained UTNet.pth model is loaded successfully...
Test Accuracy: 94.7537, Test Precision: 0.2631, Test Dice Score: 0.3384, Test IOU Score: 0.2336


## Inference Visualization of Trained Models

In [1]:
# Function to normalize the mask for better visualization
def normalize_array(array):
    array_min = array.min()
    array_max = array.max()
    normalized_array = (array - array_min) / (array_max - array_min) * 255
    return normalized_array.astype(np.uint8)

def display_inference_image_mask(images_filenames, images_dataset_path, masks_data_path, inference_masks, inference_title, font_size=10):
    # Determine the number of columns and rows and adjust figsize accordingly
    cols = len(inference_models) + 2  # 2 is added to include the ground truth image and mask along with the predicted masks of trained models
    rows = len(images_filenames)
    # Set the figsize dynamically based on the number of columns
    base_figsize_width = 10  # Base width for three columns
    base_figsize_height = 24  # Base height for the given number of rows
    extra_width_per_col = 2  # Additional width for each extra column
    # Calculate the appropriate figsize based on the number of columns
    figsize_width = base_figsize_width + (cols - 3) * extra_width_per_col
    figsize = (figsize_width, base_figsize_height)

    # Create the subplot grid with the dynamic figsize
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=figsize)

    # Loop through the image filenames and display them along with their masks
    for i, image_filename in enumerate(images_filenames):
        # Load the grounth truth image
        image_path = os.path.join(images_dataset_path, image_filename)
        image = np.array(Image.open(image_path).convert("RGB"))

        # Load the grounth truth mask
        mask_path = os.path.join(masks_data_path, image_filename.replace(".jpg", ".png"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)

        # Display the grounth image and mask
        ax[i, 0].imshow(image)
        ax[i, 1].imshow(mask, cmap='gray')
        # Set the titles with the specified font size
        ax[i, 0].set_title("Image", fontsize=font_size)
        ax[i, 1].set_title("GT Mask", fontsize=font_size)
        # Turn off axes
        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        for j, (infer_model, infer_mask) in enumerate(inference_masks.item()):
            predicted_mask = normalize_array(inference_masks[infer_model][i])  # Normalize for better color mapping
            # Display the predicted mask
            ax[i, j+2].imshow(predicted_mask, cmap='gray')
            # Set titles of predicted masks
            ax[i, j+2].set_title(inference_title[infer_model], fontsize=font_size)
            # Turn off axes
            ax[i, j+2].set_axis_off()
    
    # Adjust layout and display the plot
    plt.tight_layout()  # Adjust spacing to avoid overlaps
    plt.show()



In [46]:
# Provide the list of indices for grounth images and mask
selected_indices = [9, 11, 18, 48, 250, 500]
selected_GTimages = [test_images_filenames[i] for i in selected_indices]

# Provide the dictionary of the trained inference models for the visualization
inference_models = {"ResUNet": [infer_ResNetUNet18_AM[i] for i in selected_indices],
                    "EffNetUNet": [infer_EffNetUNetB1_AM[i] for i in selected_indices],
                    "MANet": [infer_ResMANet34[i] for i in selected_indices],
                    "DcsNet": [infer_DcsNet[i] for i in selected_indices]
                    }
# Provide titles of the inference masks from the trained models
inference_title = {"ResUNet": "ResNet34-UNet",
                    "EffNetUNet": "EfficientNetB1-UNet",
                    "MANet": "ResNet34-MANet",
                    "DcsNet": "DcsNet"
                    }

In [28]:
import numpy as np

def dummy_inference():
    return [np.random.rand(1, 256, 256) for _ in selected_indices]

inference_models = {
    "ResUNet": dummy_inference(),
    "EffNetUNet": dummy_inference(),
    "MANet": dummy_inference(),
    "DcsNet": dummy_inference()
}

print((inference_models["ResUNet"]))

AttributeError: 'list' object has no attribute 'size'

In [63]:
for j, (model_name, predicted_masks) in enumerate(inference_models.items()):
    print(j)
    print(len(predicted_masks))
    print((inference_models[model_name][j]))
    print(type(inference_title[model_name]))

0
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'str'>
1
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'str'>
2
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'str'>
3
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'str'>
